# Problem formulation

1. Define decision variables
2. Define constraints (on the decision variables)
3. Define objective function

## Global decision variables - FSI

Inputs required:

- maximum building extents in 3D (in the form of a .obj to be provided by user)
- the current occupation lattice

In [7]:
# first we define the decision variables for the programmatic minimum requirement of the entire project (manifesting itself as FSI)
# then we find the solar yield of the entire project, this includes solar heat gain and PV potential as aggregated values (manifesting itself as collisisons per m2)

import trimesh as tm
import os
import pyvista as pv
import numpy as np
import topogenesis as tg

lattice_path = os.path.relpath("../data/voxelized_envelope.csv")
occ_lattice = tg.lattice_from_csv(lattice_path) # the current occupation lattice

path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height, USER INPUT)

base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint
env_all_vox = occ_lattice.flatten() # flattened lattice of all possible positions, True is occupied, False is unoccupied
FSI_min = 2.0 # current goal for the FSI, can be a variable

### FSI ### maybe it is faster to take footprint outside of the equation so it does not recompute the footprint everytime?

def FSI(footprint, occupationlattice): # calculate FSI based on occupation lattice and mesh plot
    vox_size = occupationlattice.unit[0]
    m2_plot = footprint.area/2 # area calculates both sides of the flattened mesh, we need only 1
    cell_count = [i for i in occupationlattice if i != False ]
    m2_total = vox_size**2*len(cell_count)
    FSI = m2_total/m2_plot
    return FSI

def FSI_fast(footprint, floors, floor_percent=0.8): # calculate FSI based on floorcount, plot coverage of the floors, and mesh plot
    m2_plot = footprint.area/2 # area claculates both sides of the flattened mesh, we need only 1
    m2_floor = m2_plot*floor_percent
    m2_total = m2_floor*floors
    FSI = m2_total/m2_plot
    return FSI

FSI_fast(base,10, 0.9)

9.0

## Global decision variables - Aggregated Solar Score (ASS) (working title)

Inputs required:

- Building mass (mesh or .obj) OR floorcount + roof area
- Coordinates OR solar vectors (prepared) using Ladybug plugin

In [13]:
# this includes a. the PV potential (calculated by roof area over total floor area)
# and b. the solar heat gain over the year (calculated by G-value/SHGC, window/wall ratio, solar heat gain coefficient of opaque components)

# PV potential - how much roof area per m2 is available for installing PV

def PV_fast(floors): # this gives an ESTIMATE on the global scale. Needs to be recomputed at the end of the local scale (or continuously calculated) DO: find out
    PV = 1/floors
    return PV

def PV()


0.3333333333333333

## Local cost/objective function - Energy usage (minimize)

inputs required:

- current floor area (per building unit type)
- constant value (per building unit type)

## Local cost/objective function - daylighting (threshold)

inputs required:

- Current building unit (type)
- Surrounding voxels
- Solar path

## Visualisation

In [105]:
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter

# initiating the plotter
p = pv.Plotter(notebook=True)
occ_lattice.fast_vis(p)

# adding the base mesh: light blue
p.add_mesh(tri_to_pv(bounds), color='#abd8ff')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(285.89291929890817, 200.89291929890814, 270.89291929890817),
 (35.0, -50.0, 20.0),
 (0.0, 0.0, 1.0)]